In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [9]:
random.seed(777)
torch.manual_seed(777)

# parameters
learning_rate = 0.001
training_epochs = 10
batch_size = 100

#MNIST dataset download
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True,
                          transform=transforms.ToTensor(),
                          download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                          train = False,
                          transform=transforms.ToTensor(),
                          download=True)

# dataset loader
data_loader_xa = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

data_loader_he = data_loader_xa

# nn layers
linear1_xa = torch.nn.Linear(784, 256, bias=True)
linear2_xa = torch.nn.Linear(256, 256, bias=True)
linear3_xa = torch.nn.Linear(256, 10, bias=True)

linear1_he = torch.nn.Linear(784, 256, bias=True)
linear2_he = torch.nn.Linear(256, 256, bias=True)
linear3_he = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

# xavier initialization
torch.nn.init.xavier_uniform_(linear1_xa.weight)
torch.nn.init.xavier_uniform_(linear2_xa.weight)
torch.nn.init.xavier_uniform_(linear3_xa.weight)

# He initialization
torch.nn.init.kaiming_uniform_(linear1_he.weight)
torch.nn.init.kaiming_uniform_(linear2_he.weight)
torch.nn.init.kaiming_uniform_(linear3_he.weight)

# model
model_xa = torch.nn.Sequential(linear1_xa, relu, linear2_xa, relu, linear3_xa)
model_he = torch.nn.Sequential(linear1_he, relu, linear2_he, relu, linear3_he)

# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()   # Softmax is internally computed.
optimizer_xa = torch.optim.Adam(model_xa.parameters(), lr=learning_rate)
optimizer_he = torch.optim.Adam(model_he.parameters(), lr=learning_rate)

total_batch_xa = len(data_loader_xa)
total_batch_he = len(data_loader_he)


In [10]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader_xa:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer_xa.zero_grad()
        hypothesis = model_xa(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer_xa.step()

        avg_cost += cost / total_batch_xa

    print('!Xavier! Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader_he:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28)
        Y = Y

        optimizer_he.zero_grad()
        hypothesis = model_he(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer_he.step()

        avg_cost += cost / total_batch_he

    print('!He! Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

!Xavier! Epoch: 0001 cost = 0.236406788
!Xavier! Epoch: 0002 cost = 0.088970706
!Xavier! Epoch: 0003 cost = 0.059096497
!Xavier! Epoch: 0004 cost = 0.042276364
!Xavier! Epoch: 0005 cost = 0.032123659
!Xavier! Epoch: 0006 cost = 0.025813062
!Xavier! Epoch: 0007 cost = 0.021159587
!Xavier! Epoch: 0008 cost = 0.018840605
!Xavier! Epoch: 0009 cost = 0.015069388
!Xavier! Epoch: 0010 cost = 0.013015751
Learning finished
!He! Epoch: 0001 cost = 0.235212296
!He! Epoch: 0002 cost = 0.090423077
!He! Epoch: 0003 cost = 0.058507182
!He! Epoch: 0004 cost = 0.040901314
!He! Epoch: 0005 cost = 0.030456509
!He! Epoch: 0006 cost = 0.024937019
!He! Epoch: 0007 cost = 0.020377941
!He! Epoch: 0008 cost = 0.016706130
!He! Epoch: 0009 cost = 0.016230999
!He! Epoch: 0010 cost = 0.011439699
Learning finished


In [11]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    print('!Xavier!')
    prediction_xa = model_xa(X_test)
    correct_prediction = torch.argmax(prediction_xa, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]

    print('Label: ', Y_single_data.item())
    single_prediction = model_xa(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())
    
    print('!He!')
    prediction_he = model_he(X_test)
    correct_prediction = torch.argmax(prediction_he, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]

    print('Label: ', Y_single_data.item())
    single_prediction = model_he(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

C:\Users\JH PARK\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\JH PARK\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


!Xavier!
Accuracy: 0.9811999797821045
Label:  8
Prediction:  8
!He!
Accuracy: 0.9807999730110168
Label:  5
Prediction:  5
